In [1]:
] activate "/Users/anarres/Documents/projects/BioXP"

 Activating new environment at `/Users/anarres/Documents/projects/BioXP/Project.toml`


In [3]:
using BioXP
using ProgressMeter
using BenchmarkTools
# using BenchmarkTools

┌ Info: Precompiling ProgressMeter [92933f4c-e287-5a05-a399-4b506db050ca]
└ @ Base loading.jl:1278
┌ Info: Precompiling BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1278


### Check nthreads

In [8]:
Threads.nthreads()

1

### Put everything in one function for benchmarking

In [40]:
function run_all()
    ## Organism JSONs from ecg (JGI)
    # input_dir = "data/input/rids/"
    input_dir = "../test/data/rids-methanogens/"

    ## Master file from ecg (with dgs added from `add_dgs_to_master` .py files)
    rstructs_path = "../test/data/master_from_redges-og-submission.json"

    ## User defined seeds/targets
    seeds_path = "../data/seeds/encel_papers_2019.json"
    targets_path = "../data/seeds/seeds.json"
    sid_name = "Contains KEGGID P"
    tid_name = "targets_Freilich09";

    ## Path to write to
    # write_dir = joinpath("data","output",sid_name)
    write_dir = joinpath("data","output",sid_name*"-methanogens")

    if !ispath(write_dir)
        mkpath(write_dir)
    end
    
    sids = readkeyedids(seeds_path)[sid_name]
    tids = readkeyedids(targets_path)[tid_name]
    rstructs = readmaster(rstructs_path)

#     for path in readdir(input_dir)

#         org_dir = joinpath(input_dir,path)

#         if isdir(org_dir) &  !startswith(path,".") 

#             if !ispath(joinpath(write_dir,path))
#                 mkpath(joinpath(write_dir,path))
#             end

    p = Progress(length(readdir(input_dir)),desc="$input_dir")

#     println(org_dir)
    Threads.@threads for (i,fname) in collect(enumerate(readdir(input_dir)))
#             Random.seed!(seedoffset+i)
        if !startswith(fname,".") 
            rids = readids(joinpath(input_dir,fname))
            write_path = joinpath(write_dir,basename(input_dir),fname)
            x, t, cids, X, Y = expand(rstructs,rids,sids,tids=tids,write_path=write_path) ## new results
            next!(p)
        end
    end
    
    simple_output_dir = write_dir #joinpath("data","output",sid_name)
    
    formatbioxpoutput(simple_output_dir)
    
#     for path in readdir(simple_output_dir)
# #         org_dir = joinpath(simple_output_dir,path)
#         @show path
#         if !startswith(path,".") & !isdir(path) 
#             formatbioxpoutput(joinpath(simple_output_dir,path))
#         end
#     end
end

run_all (generic function with 1 method)

In [41]:
if true & false
    println(1)
end

In [42]:
@benchmark run_all()

../test/data/rids-methanogens/ 86%|██████████████████   |  ETA: 0:00:00

BenchmarkTools.Trial: 
  memory estimate:  233.30 MiB
  allocs estimate:  562073
  --------------
  minimum time:     1.036 s (3.54% GC)
  median time:      1.080 s (4.31% GC)
  mean time:        1.093 s (7.90% GC)
  maximum time:     1.162 s (13.76% GC)
  --------------
  samples:          5
  evals/sample:     1

Benchmarking "Contains KEGGID P-Methanogens" without sparse Matrices:

```
BenchmarkTools.Trial: 
  memory estimate:  233.30 MiB
  allocs estimate:  562073
  --------------
  minimum time:     1.036 s (3.54% GC)
  median time:      1.080 s (4.31% GC)
  mean time:        1.093 s (7.90% GC)
  maximum time:     1.162 s (13.76% GC)
  --------------
  samples:          5
  evals/sample:     1
```

### Inputs

In [5]:
## Organism JSONs from ecg (JGI)
# input_dir = "data/input/rids/"
input_dir = "../test/data/input/rids-methanogens/"

## Master file from ecg (with dgs added from `add_dgs_to_master` .py files)
rstructs_path = "../test/data/input/rstructs/master_from_redges-og-submission.json"

## User defined seeds/targets
seeds_path = "../data/input/seeds/encel_papers_2019.json"
targets_path = "../data/input/seeds/seeds.json"
sid_name = "Contains KEGGID P"
tid_name = "targets_Freilich09";

## Path to write to
# write_dir = joinpath("data","output",sid_name)
write_dir = joinpath("data","output",sid_name*"-methanogens")

if !ispath(write_dir)
    mkpath(write_dir)
end

"data/output/Contains KEGGID P-methanogens"

### Run expansion

Double check number of accessible threads

In [6]:
Threads.nthreads()

1

Looping such that seeds are logically assigned

In [10]:
sids = readkeyedids(seeds_path)[sid_name]
tids = readkeyedids(targets_path)[tid_name]
rstructs = readmaster(rstructs_path)

for path in readdir(input_dir)
    
    org_dir = joinpath(input_dir,path)
    
    if isdir(org_dir) &  !startswith(path,".") 
        
        if !ispath(joinpath(write_dir,path))
            mkpath(joinpath(write_dir,path))
        end
        
        p = Progress(length(readdir(org_dir)),desc="$path")
        
        println(org_dir)
        Threads.@threads for (i,fname) in collect(enumerate(readdir(org_dir)))
#             Random.seed!(seedoffset+i)
            if !startswith(fname,".") 
                rids = readids(joinpath(org_dir,fname))
                write_path = joinpath(write_dir,basename(org_dir),fname)
                x, t, cids, X, Y = expand(rstructs,rids,sids,tids,write_path) ## new results
                next!(p)
            end
        end
    end
end

data/input/rids-methanogens/archaea


archaea 50%|██████████████████████                      |  ETA: 0:00:00

### Format output

In [11]:
simple_output_dir = write_dir #joinpath("data","output",sid_name)
    
for path in readdir(simple_output_dir)
    org_dir = joinpath(simple_output_dir,path)
    if !startswith(path,".") 
        formatbioxpoutput(org_dir)
    end
end